## Tarea 2
- Usar anomalies python package
- Implementar multiprocessing para calcular las soluciones

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np

from anomalies import anomaly
from lk import generate_lk

import pandas as pd
import multiprocessing as multiprocessing
import dask.array as da


In [2]:

def compute_z(lk, zmax=32):
    
    n = len(lk)
    
    l, k = lk[:int(n/2)].flatten(), lk[int(n/2):].flatten()
    
    z_ = anomaly.free(l, k)

    if z_[0]<0:
        z_=(-1)*z_

    gcd = np.gcd.reduce(z_)

    if gcd==0:  
        z = z_
    else:
        z = (z_/gcd).astype(int)

    
    if (0 in z):
        
        return {}
    
    elif (np.abs(z).max() > zmax):
        
        return {}
    
    elif np.unique(np.abs(z)).shape != np.unique(z).shape:
    
        return {}
    
    else:

        
        
        results = {
            'l': l.tolist(),
            'k': k.tolist(),
            'z': z.tolist(),
            'gcd': gcd,
        }   
    
        return results


In [3]:
def find_all_z(n, lkmax, N):

    lk = generate_lk(n, lkmax, N)

    pool = multiprocessing.Pool()
    results = pool.map(compute_z, lk)

    return results


In [4]:
def solution_set(n, lkmax, N):
    
    zs = find_all_z(n, lkmax, N)

    all_zs = [d for d in zs if d]

    df = pd.DataFrame(all_zs)
    df = df.sort_values(by = ['gcd'], ignore_index=True)

    df['copy'] = df['z'].astype(str)
    df = df.drop_duplicates('copy').drop('copy', axis='columns').reset_index(drop=True)

    return df    

In [5]:
df_5 = solution_set(5, 6, 500000)
df_5

,l,k,z,gcd
0,[-2],"[1, 2]","[2, 4, -7, -9, 10]",1
1,[-1],"[1, -5]","[2, 18, -23, -25, 28]",1
2,[4],"[-1, 2]","[4, 9, -14, -25, 26]",1
3,[5],"[1, -2]","[7, 9, -20, -22, 26]",2
4,[-6],"[-1, 1]","[1, 14, -17, -18, 20]",2
5,[-5],"[1, 2]","[5, 8, -14, -26, 27]",2
6,[-1],"[-2, 3]","[7, 13, -25, -27, 32]",2
7,[4],"[-1, 1]","[1, 5, -7, -8, 9]",2
8,[3],"[1, -4]","[7, 8, -17, -25, 27]",2
9,[-2],"[1, -5]","[5, 6, -12, -21, 22]",2


In [6]:
df_6 = solution_set(6, 11, 5000000)
df_6

,l,k,z,gcd
0,"[1, 2]","[1, 0]","[1, -2, -3, 5, 5, -6]",1
1,"[2, 1]","[-3, -1]","[1, 5, 6, -14, -19, 21]",1
2,"[1, -2]","[3, 0]","[7, -10, -11, 15, 21, -22]",1
3,"[-2, -1]","[-6, -9]","[2, 4, 6, -15, -17, 20]",1
4,"[1, -1]","[-4, 3]","[2, -3, -8, 10, 12, -13]",1
...,...,...,...,...
137,"[-4, -8]","[-5, 3]","[19, -20, -24, 26, 29, -30]",32
138,"[-3, 3]","[-9, 7]","[9, -11, -19, 23, 27, -29]",36
139,"[-3, 3]","[-9, 1]","[5, -9, -13, 19, 25, -27]",36
140,"[6, -3]","[9, 8]","[17, -18, -22, 25, 25, -27]",54
